In [4]:
# !wget -c https://cdn-datasets.huggingface.co/EsperBERTo/data/oscar.eo.txt

## BPE：Byte Pair Encoding

- BPE：Byte pair encoding，字节对编码；
    - ASCII中，一个字符（Character）对应一个字节（Byte）。
    - 在处理英语等西方语言时，字符和字节的对应关系较为简单，一个字符通常是一个字节。
    - 在处理中文等东亚语言时，字符和字节的对应关系较为复杂，一个字符可能是多个字节。
- Byte-Pair Encoding (BPE) was initially developed as an algorithm to **compress texts** (and encoding to token ids), and then **used by OpenAI** for tokenization when pretraining the GPT model. It’s used by a lot of Transformer models, including GPT, GPT-2, RoBERTa, BART, and DeBERTa.


```
# frequencies
("hug", 10), ("pug", 5), ("pun", 12), ("bun", 4), ("hugs", 5)

# split to characters, form the initial vocabulary
("h" "u" "g", 10), ("p" "u" "g", 5), ("p" "u" "n", 12), ("b" "u" "n", 4), ("h" "u" "g" "s", 5)

# adj char => 2 chars
# ug: 10+5+5 = 20
Vocabulary: ["b", "g", "h", "n", "p", "s", "u", "ug"]
Corpus: ("h" "ug", 10), ("p" "ug", 5), ("p" "u" "n", 12), ("b" "u" "n", 4), ("h" "ug" "s", 5)

# u + n => 12+4
Vocabulary: ["b", "g", "h", "n", "p", "s", "u", "ug", "un"]
Corpus: ("h" "ug", 10), ("p" "ug", 5), ("p" "un", 12), ("b" "un", 4), ("h" "ug" "s", 5)

# h + ug => 10 + 5
Vocabulary: ["b", "g", "h", "n", "p", "s", "u", "ug", "un", "hug"]
Corpus: ("hug", 10), ("p" "ug", 5), ("p" "un", 12), ("b" "un", 4), ("hug" "s", 5)
```

In [20]:
corpus = [
    "This is the Hugging Face Course.",
    "This chapter is about tokenization.",
    "This section shows several tokenizer algorithms.",
    "Hopefully, you will be able to understand how they are trained and generate tokens.",
]

In [21]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("gpt2")

In [25]:
tokenizer.backend_tokenizer.pre_tokenizer.pre_tokenize_str(corpus[0])

[('This', (0, 4)),
 ('Ġis', (4, 7)),
 ('Ġthe', (7, 11)),
 ('ĠHugging', (11, 19)),
 ('ĠFace', (19, 24)),
 ('ĠCourse', (24, 31)),
 ('.', (31, 32))]

In [32]:
from collections import defaultdict

word_freqs = defaultdict(int)

for text in corpus:
    words_with_offsets = tokenizer.backend_tokenizer.pre_tokenizer.pre_tokenize_str(text)
    new_words = [word for word, offset in words_with_offsets]
    for word in new_words:
        word_freqs[word] += 1

# word_freqs

In [33]:
alphabet = []

for word in word_freqs.keys():
    for letter in word:
        if letter not in alphabet:
            alphabet.append(letter)
alphabet.sort()
# alphabet

In [34]:
vocab = ["<|endoftext|>"] + alphabet.copy()
# vocab

In [35]:
splits = {word: [c for c in word] for word in word_freqs.keys()}
# splits

In [36]:
def compute_pair_freqs(splits):
    pair_freqs = defaultdict(int)
    for word, freq in word_freqs.items():
        split = splits[word]
        if len(split) == 1:
            continue
        for i in range(len(split) - 1):
            pair = (split[i], split[i + 1])
            pair_freqs[pair] += freq
    return pair_freqs

In [37]:
pair_freqs = compute_pair_freqs(splits)

for i, key in enumerate(pair_freqs.keys()):
    print(f"{key}: {pair_freqs[key]}")
    if i >= 5:
        break

('T', 'h'): 3
('h', 'i'): 3
('i', 's'): 5
('Ġ', 'i'): 2
('Ġ', 't'): 7
('t', 'h'): 3


In [38]:
best_pair = ""
max_freq = None

for pair, freq in pair_freqs.items():
    if max_freq is None or max_freq < freq:
        best_pair = pair
        max_freq = freq

print(best_pair, max_freq)

('Ġ', 't') 7


In [39]:
merges = {("Ġ", "t"): "Ġt"}
vocab.append("Ġt")

In [40]:
vocab

['<|endoftext|>',
 ',',
 '.',
 'C',
 'F',
 'H',
 'T',
 'a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'r',
 's',
 't',
 'u',
 'v',
 'w',
 'y',
 'z',
 'Ġ',
 'Ġt']

In [41]:
def merge_pair(a, b, splits):
    for word in word_freqs:
        split = splits[word]
        if len(split) == 1:
            continue

        i = 0
        while i < len(split) - 1:
            if split[i] == a and split[i + 1] == b:
                split = split[:i] + [a + b] + split[i + 2 :]
            else:
                i += 1
        splits[word] = split
    return splits

In [43]:
splits = merge_pair("Ġ", "t", splits)
splits

{'This': ['T', 'h', 'i', 's'],
 'Ġis': ['Ġ', 'i', 's'],
 'Ġthe': ['Ġt', 'h', 'e'],
 'ĠHugging': ['Ġ', 'H', 'u', 'g', 'g', 'i', 'n', 'g'],
 'ĠFace': ['Ġ', 'F', 'a', 'c', 'e'],
 'ĠCourse': ['Ġ', 'C', 'o', 'u', 'r', 's', 'e'],
 '.': ['.'],
 'Ġchapter': ['Ġ', 'c', 'h', 'a', 'p', 't', 'e', 'r'],
 'Ġabout': ['Ġ', 'a', 'b', 'o', 'u', 't'],
 'Ġtokenization': ['Ġt',
  'o',
  'k',
  'e',
  'n',
  'i',
  'z',
  'a',
  't',
  'i',
  'o',
  'n'],
 'Ġsection': ['Ġ', 's', 'e', 'c', 't', 'i', 'o', 'n'],
 'Ġshows': ['Ġ', 's', 'h', 'o', 'w', 's'],
 'Ġseveral': ['Ġ', 's', 'e', 'v', 'e', 'r', 'a', 'l'],
 'Ġtokenizer': ['Ġt', 'o', 'k', 'e', 'n', 'i', 'z', 'e', 'r'],
 'Ġalgorithms': ['Ġ', 'a', 'l', 'g', 'o', 'r', 'i', 't', 'h', 'm', 's'],
 'Hopefully': ['H', 'o', 'p', 'e', 'f', 'u', 'l', 'l', 'y'],
 ',': [','],
 'Ġyou': ['Ġ', 'y', 'o', 'u'],
 'Ġwill': ['Ġ', 'w', 'i', 'l', 'l'],
 'Ġbe': ['Ġ', 'b', 'e'],
 'Ġable': ['Ġ', 'a', 'b', 'l', 'e'],
 'Ġto': ['Ġt', 'o'],
 'Ġunderstand': ['Ġ', 'u', 'n', 'd', 'e', 'r', '

In [44]:
vocab_size = 50

while len(vocab) < vocab_size:
    pair_freqs = compute_pair_freqs(splits)
    best_pair = ""
    max_freq = None
    for pair, freq in pair_freqs.items():
        if max_freq is None or max_freq < freq:
            best_pair = pair
            max_freq = freq
    splits = merge_pair(*best_pair, splits)
    merges[best_pair] = best_pair[0] + best_pair[1]
    vocab.append(best_pair[0] + best_pair[1])

In [45]:
merges

{('Ġ', 't'): 'Ġt',
 ('i', 's'): 'is',
 ('e', 'r'): 'er',
 ('Ġ', 'a'): 'Ġa',
 ('Ġt', 'o'): 'Ġto',
 ('e', 'n'): 'en',
 ('T', 'h'): 'Th',
 ('Th', 'is'): 'This',
 ('o', 'u'): 'ou',
 ('s', 'e'): 'se',
 ('Ġto', 'k'): 'Ġtok',
 ('Ġtok', 'en'): 'Ġtoken',
 ('n', 'd'): 'nd',
 ('Ġ', 'is'): 'Ġis',
 ('Ġt', 'h'): 'Ġth',
 ('Ġth', 'e'): 'Ġthe',
 ('i', 'n'): 'in',
 ('Ġa', 'b'): 'Ġab',
 ('Ġtoken', 'i'): 'Ġtokeni'}

In [46]:
vocab

['<|endoftext|>',
 ',',
 '.',
 'C',
 'F',
 'H',
 'T',
 'a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'r',
 's',
 't',
 'u',
 'v',
 'w',
 'y',
 'z',
 'Ġ',
 'Ġt',
 'is',
 'er',
 'Ġa',
 'Ġto',
 'en',
 'Th',
 'This',
 'ou',
 'se',
 'Ġtok',
 'Ġtoken',
 'nd',
 'Ġis',
 'Ġth',
 'Ġthe',
 'in',
 'Ġab',
 'Ġtokeni']

### tokenize a text

In [48]:
merges

{('Ġ', 't'): 'Ġt',
 ('i', 's'): 'is',
 ('e', 'r'): 'er',
 ('Ġ', 'a'): 'Ġa',
 ('Ġt', 'o'): 'Ġto',
 ('e', 'n'): 'en',
 ('T', 'h'): 'Th',
 ('Th', 'is'): 'This',
 ('o', 'u'): 'ou',
 ('s', 'e'): 'se',
 ('Ġto', 'k'): 'Ġtok',
 ('Ġtok', 'en'): 'Ġtoken',
 ('n', 'd'): 'nd',
 ('Ġ', 'is'): 'Ġis',
 ('Ġt', 'h'): 'Ġth',
 ('Ġth', 'e'): 'Ġthe',
 ('i', 'n'): 'in',
 ('Ġa', 'b'): 'Ġab',
 ('Ġtoken', 'i'): 'Ġtokeni'}

In [47]:
def tokenize(text):
    pre_tokenize_result = tokenizer._tokenizer.pre_tokenizer.pre_tokenize_str(text)
    pre_tokenized_text = [word for word, offset in pre_tokenize_result]
    splits = [[l for l in word] for word in pre_tokenized_text]
    for pair, merge in merges.items():
        for idx, split in enumerate(splits):
            i = 0
            while i < len(split) - 1:
                if split[i] == pair[0] and split[i + 1] == pair[1]:
                    split = split[:i] + [merge] + split[i + 2 :]
                else:
                    i += 1
            splits[idx] = split

    return sum(splits, [])

## train a tokenizer

- https://colab.research.google.com/github/huggingface/blog/blob/main/notebooks/01_how_to_train.ipynb#scrollTo=hO5M3vrAhcuj

In [5]:
!pip list | grep -E 'transformers|tokenizers'

sentence-transformers                   2.2.2
tokenizers                              0.19.1
transformers                            4.42.3


In [7]:
from tokenizers import ByteLevelBPETokenizer
from pathlib import Path

In [11]:
tokenizer = ByteLevelBPETokenizer()
paths = ['./oscar.eo.txt']

In [12]:
# cpu 
tokenizer.train(files=paths, vocab_size=52_000, min_frequency=2, special_tokens=[
    "<s>",
    "<pad>",
    "</s>",
    "<unk>",
    "<mask>",
])

In [14]:
tokenizer.save_model("EsperBERTo")

# tokenizer = ByteLevelBPETokenizer(
#     "./EsperBERTo/vocab.json",
#     "./EsperBERTo/merges.txt",
# )

['EsperBERTo/vocab.json', 'EsperBERTo/merges.txt']

In [18]:
tokenizer.token_to_id('</s>')

2

In [19]:
tokenizer.encode('</s>').tokens

['</s>']